Corriger une prédiction HTR
===
<br>
D'après la démarche expliquée dans la documentation du projet DAHN, on procède selon les étapes suivantes.

On privilégie le format XML-Page pour exporter et réimporter les prédictions HTR dans le logiciel de transcription.

In [1]:
from py.constantes import XMLaCORRIGER, XMLCORRIGEES
print(f"On place les prédictions HTR dans le dossier : {XMLaCORRIGER}")

On place les prédictions HTR dans le dossier : ./xmlPage-aCorriger/


# Analyser le contenu de la prédiction

Exécuter le script **spellcheck_texts_PAGEXML.py**, qui analyse chaque mot des prédictions HTR et génère pour chaque fichier XML à corriger un dictionnaire au format Json contenant des propositions de corrections : 

In [22]:
!python3 py/spellcheckTexts.py

Le fichier ./xmlPage-aCorriger/CdS02_Konv002-03_0056.xml est en cours de lecture.
=> Le dictionnaire ./py/dicos/page_CdS02_Konv002-03_0056.json a été écrit correctement.



# Corriger les dictionnaires de pages

Corriger à la main les entrées du dictionnaire de chacun des fichiers commençant par "page" générés dans le dossier [dicos/](py/dicos/).

**Attention** : il faut bien veiller à…
- **Ne pas transformer de mots qui seraient justes dans un autre contexte** (comme le mot **"tout"** dans l'exemple suivant, qui ne doit pas être corrigé en "août" bien que le contexte s'y prête, au risque de modifier des "tout" pertinents dans leur propre contexte)

- **Éliminer les propositions de lemmes erronées**, en modifiant le lemme comme "null" (**"Paris"** ne doit pas être corrigé en "pris" dans l'exemple suivant)

Il n'est pas nécessaire de transformer les mots justes dont le lemme est indiqué comme "null", car ils seront intégrés au réservoir des mots connus dès leurs prochaine rencontre (ici l'exemple de **"homme"**).

```json
{
    "tout": {
      "lem": null,
      "ctxt": "Dyck, ce 3 TOUT 1818"
   },
    "Paris": {
      "lem": "pris",
      "ctxt": "c est qu il retournera à PARIS dans le mois"
   },
    "homme": {
      "lem": null,
      "ctxt": "que ce brave HOMME se rétablisse"
   },
}
```

Les **patronymes** représentent une difficulté importante de la transcription. Ils ont par bonheur été déjà identifiés. On se reportera donc aux notices du site web. On se reportera aux [outils du dossier donnees/](https://github.com/sbiay/CdS-edition/tree/main/donnees#outils), et plus particulièrement au [notebook](https://github.com/sbiay/CdS-edition/blob/main/donnees/Obtenir_metadonnees_images.ipynb) dédié à la récupération des liens vers les notices de l'inventaire à partir d'un nom de fichier image.

D'une façon générale, il est peu efficace de proposer des corrections pour les mots brefs et rencontrés dans **les passages les plus cursifs** (bouts de lignes, date, expéditeur) : la pércision de la reconnaissance y est faible, et plus un mot et bref plus la correction pose de problèmes d'ambiguïté.

A l'inverse, il n'est pas gênant de proposer des corrections pour les mots très mal prédits, longs, coupés par une espace. L'intérêt d'inscrire cette correction dans le dictionnaire est limité, car la probabilité de recroiser la même forme est à peu près nulle, mais le risque d'ambiguïté est tout aussi faible : autant appliquer la correction dès le stade de la validation du dictionnaire de page plutôt que d'attendre de la recroiser à la relecture finale.

# Enrichir le dictionnaire du projet

Une fois cette tâche accomplie, on applique le script [integrationDicts.py](./py/integrationDicts.py). Voyons dans un premier temps son rôle et les options qu'il propose :

In [1]:
!python3 py/integrationDicts.py --help

Usage: integrationDicts.py [OPTIONS]

  Ce script prend comme paramètres une source consistant en un fichier Json
  d'entrée contenu dans le dossier dicos le compare au contenu de
  correctionsCDS.json, intègre les entrées nouvelles et retourne des
  messages d'alertes pour les entrées générant un conflit d'intégration.
  :param fichierdictpages: fichier Json issu de la correction automatisée
  d'une page de transcription :type fichierdictpages: Json :returns: None

Options:
  -f, --file TEXT  Nom de fichier contenu dans le dossier ./py/dicos/
  -A, --all        Prend tous les fichiers du dossier ./py/dicos/
  --help           Show this message and exit.


In [23]:
# Ajouter l'option qui convient et relancer le script
!python3 py/integrationDicts.py -A

Traitement du fichier page_CdS02_Konv002-03_0056.json

=> Le dictionnaire correctionsCDS.json est désormais à jour.


Si des problèmes d'ambiguïté ont été signalés, on doit les corriger à la main dans les dictionnaires. Deux cas se présentent :
- Si l'on souhaite faire prévaloir la correction proposée dans le dictionnaire complet de la correspondance [correctionsCDS.json](./py/dicos/correctionsCDS.json), on inscrit "null" comme lemme dans le dictionaire de la page
- Si l'on considère qu'il ne faut plus proposer de correction pour la forme signalée, qu'elle doit être considérée comme ambiguë, on inscrit "null" comme lemme dans le dictionaire de la page **et aussi** comme lemme de la forme dans le [correctionsCDS.json](./py/dicos/correctionsCDS.json).

On relance le script après chaque correction que l'on effectue.

# Appliquer le dictionnaire du projet aux fichiers XML

On procède grâce au script [text_correction_XML.py](./py/text_correction_XML.py) :

In [24]:
!python3 py/textCorrection.py

Le fichier CdS02_Konv002-03_0056.xml a été corrigé avec succès


On finalise le processus de correction avec les étapes suivantes :
1. Réimporter les fichiers corrigés dans eScriptorium ;
2. Finaliser manuellement la correction des transcriptions ;
3. Télécharger la verité de terrain pour la sauvegarder au format XML-Alto.

In [ ]:
print(f"On place les vérités de terrain dans le dossier : {XMLCORRIGEES}")